In [ ]:
import pickle
import os
import pandas as pd
import matplotlib.pyplot as plt

In [ ]:
# Adjust these for the system you are running on

checkpoint_path = '/raid/msheller/fets_challenge/workspace/checkpoint'
experiments = [
    21,
    22,
    20,
    4,
    23,
    24,
    27,
    25,
    28,
    26,
]

In [ ]:
def load_results_dataframe(experiment_number):
    global checkpoint_path

    state_path = os.path.join(checkpoint_path, 'experiment_{}'.format(experiment_number), 'state.pkl')

    with open(state_path, 'rb') as f:
        results = pickle.load(f)

    collaborator_names, round_num, collaborator_time_stats, total_simulated_time, \
                         best_dice, best_dice_over_time_auc, collaborators_chosen_each_round, \
                         collaborator_times_per_round, experiment_results, summary = results
    
    return pd.DataFrame(data=experiment_results)

In [ ]:
dfs = {ex: load_results_dataframe(ex) for ex in experiments}

In [ ]:
def get_best_round(df, round_to=6):
    # we round because I have found rounding errors in the least-significant digits that throw off argmax
    return df.convergence_score.round(round_to).argmax()

In [ ]:
df_rows = []
for ex, df in dfs.items():
    best = get_best_round(df)
    d = df.iloc[best:best+1]
    d['experiment'] = ex
    d['Total Sim Days'] = (df['time'].max() / 86400).round(3)
    d['Total Rounds'] = df['round'].max()
    df_rows.append(d)
df = pd.concat(df_rows)
df = df.set_index('experiment')
df